# 11 - Runnable 基础概念

## 用途
学习 LangChain 1.0 的核心抽象 Runnable，理解链式拼接的基础概念和实现方法

## 学习目标
- 理解 Runnable 抽象概念和设计原理
- 掌握基础链式拼接方法
- 能构建简单的通义千问处理流程
- 掌握 invoke 方法的使用

## 🔑 前置要求
**注意**：需要先完成 OpenAI 基础调用学习，配置好 API 密钥

## 代码块独立性说明
**注意**：每个代码块都是独立的，包含完整的导入和初始化，确保可以单独运行。

## 1. Runnable 抽象概念理解

In [ ]:
# Runnable 抽象概念理解 - 独立代码块
from langchain_core.runnables import Runnable
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("🔧 Runnable 抽象概念理解:")
print("=" * 50)

# 检查 OpenAI API 配置
openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ 未找到 OPENAI_API_KEY 环境变量")
    print("   请在 .env 文件中添加: OPENAI_API_KEY=your_key_here")
else:
    print(f"✅ OpenAI API Key 已配置 (长度: {len(openai_api_key)})")
    
    print(f"\n📝 Runnable 核心概念:")
    print(f"   1. Runnable 是 LangChain 1.0 的核心抽象")
    print(f"   2. 所有组件都实现 Runnable 接口")
    print(f"   3. 支持链式组合和管道操作")
    print(f"   4. 统一的 invoke/stream/batch 方法")
    
    # 创建基础 Runnable 对象
    print(f"\n🏗️  创建基础 Runnable 对象:")
    
    # 1. ChatOpenAI 作为 Runnable
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0.7,
        max_tokens=100
    )
    
    print(f"   LLM 类型: {type(llm)}")
    print(f"   是否为 Runnable: {isinstance(llm, Runnable)}")
    print(f"   可用方法: {[method for method in dir(llm) if not method.startswith('_')]}")
    
    # 2. 简单函数作为 Runnable
    from langchain_core.runnables import RunnableLambda
    
    def simple_function(text: str) -> str:
        return f"处理结果: {text.upper()}"
    
    runnable_func = RunnableLambda(simple_function)
    print(f"\n   函数 Runnable 类型: {type(runnable_func)}")
    print(f"   是否为 Runnable: {isinstance(runnable_func, Runnable)}")
    
    # 3. 测试基础 invoke 方法
    print(f"\n🧪 测试基础 invoke 方法:")
    
    # 测试函数 Runnable
    test_input = "hello runnable"
    func_result = runnable_func.invoke(test_input)
    print(f"   函数输入: {test_input}")
    print(f"   函数输出: {func_result}")
    
    # 验证点：基础 Runnable 概念理解正确
    assert isinstance(llm, Runnable), "ChatOpenAI 不是 Runnable"
    assert isinstance(runnable_func, Runnable), "函数包装不是 Runnable"
    assert func_result == "处理结果: HELLO RUNNABLE", "函数调用结果错误"
    print(f"\n✅ 验证通过：Runnable 抽象概念理解正确")

## 2. 基础链式拼接方法

In [ ]:
# 基础链式拼接方法 - 独立代码块
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("⛓️  基础链式拼接方法测试:")
print("=" * 50)

openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ OpenAI API Key 未配置")
else:
    try:
        # 创建基础组件
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            max_tokens=50
        )
        
        # 创建简单的处理函数
        def add_prefix(text: str) -> str:
            return f"前缀: {text}"
        
        def add_suffix(text: str) -> str:
            return f"{text} :后缀"
        
        # 包装为 Runnable
        prefix_runnable = RunnableLambda(add_prefix)
        suffix_runnable = RunnableLambda(add_suffix)
        
        print(f"📝 创建的 Runnable 组件:")
        print(f"   1. LLM Runnable: {type(llm).__name__}")
        print(f"   2. 前缀处理: {type(prefix_runnable).__name__}")
        print(f"   3. 后缀处理: {type(suffix_runnable).__name__}")
        
        # 方法1: 使用 pipe 操作符
        print(f"\n🔗 方法1: 使用 pipe 操作符:")
        chain1 = prefix_runnable | suffix_runnable
        
        print(f"   链类型: {type(chain1)}")
        print(f"   链组件数量: {len(chain1.steps) if hasattr(chain1, 'steps') else '未知'}")
        
        # 测试链式调用
        test_input = "中间内容"
        result1 = chain1.invoke(test_input)
        print(f"   输入: {test_input}")
        print(f"   输出: {result1}")
        
        # 方法2: 使用 pipe 方法
        print(f"\n🔗 方法2: 使用 pipe 方法:")
        chain2 = prefix_runnable.pipe(suffix_runnable)
        
        result2 = chain2.invoke(test_input)
        print(f"   输入: {test_input}")
        print(f"   输出: {result2}")
        
        # 方法3: 多重链式拼接
        print(f"\n🔗 方法3: 多重链式拼接:")
        def process_step1(text: str) -> str:
            return f"步骤1: {text}"
        
        def process_step2(text: str) -> str:
            return f"步骤2: {text}"
        
        def process_step3(text: str) -> str:
            return f"步骤3: {text}"
        
        step1 = RunnableLambda(process_step1)
        step2 = RunnableLambda(process_step2)
        step3 = RunnableLambda(process_step3)
        
        multi_chain = step1 | step2 | step3
        multi_result = multi_chain.invoke("原始数据")
        
        print(f"   多重链输入: 原始数据")
        print(f"   多重链输出: {multi_result}")
        
        # 验证点：链式拼接方法正确
        assert result1 == "前缀: 中间内容 :后缀", "pipe 操作符结果错误"
        assert result2 == "前缀: 中间内容 :后缀", "pipe 方法结果错误"
        assert "步骤1:" in multi_result and "步骤2:" in multi_result and "步骤3:" in multi_result, "多重链结果错误"
        print(f"\n✅ 验证通过：基础链式拼接方法正确")
        
    except Exception as e:
        print(f"\n❌ 链式拼接测试失败: {e}")

## 3. prompt → llm 简单链构建

In [ ]:
# prompt → llm 简单链构建 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("🤖 prompt → llm 简单链构建测试:")
print("=" * 50)

openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ OpenAI API Key 未配置")
else:
    try:
        # 创建 LLM
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            max_tokens=60
        )
        
        # 创建 PromptTemplate
        prompt = PromptTemplate(
            input_variables=["topic"],
            template="请简单介绍一下 {topic}，用一句话概括。"
        )
        
        print(f"📝 创建的组件:")
        print(f"   1. LLM: {type(llm).__name__}")
        print(f"   2. Prompt: {type(prompt).__name__}")
        print(f"   3. 模板变量: {prompt.input_variables}")
        print(f"   4. 模板内容: {prompt.template}")
        
        # 构建 prompt → llm 链
        print(f"\n🔗 构建 prompt → llm 链:")
        chain = prompt | llm
        
        print(f"   链类型: {type(chain)}")
        print(f"   链是否可用: {chain is not None}")
        
        # 测试链式调用
        print(f"\n🧪 测试链式调用:")
        
        test_topics = [
            "人工智能",
            "机器学习", 
            "深度学习"
        ]
        
        for i, topic in enumerate(test_topics, 1):
            print(f"\n   测试 {i}: 主题 = {topic}")
            
            # 调用链
            result = chain.invoke({"topic": topic})
            
            print(f"   输入: {{'topic': '{topic}'}}")
            print(f"   输出类型: {type(result)}")
            print(f"   输出内容: {result.content}")
            
            # 验证输出格式
            assert hasattr(result, 'content'), "输出缺少 content 属性"
            assert len(result.content) > 0, "输出内容为空"
            
        # 测试错误处理
        print(f"\n🧪 测试错误处理:")
        
        try:
            # 缺少必需参数
            error_result = chain.invoke({"wrong_param": "test"})
            print(f"   ⚠️  错误输入居然成功了: {error_result}")
        except Exception as e:
            print(f"   ✅ 正确捕获错误: {type(e).__name__}")
        
        # 验证点：prompt → llm 链正确构建
        final_test = chain.invoke({"topic": "Python"})
        assert hasattr(final_test, 'content'), "最终测试输出格式错误"
        assert "Python" in final_test.content or "python" in final_test.content.lower(), "输出内容不相关"
        
        print(f"\n✅ 验证通过：prompt → llm 简单链构建正确")
        
    except Exception as e:
        print(f"\n❌ prompt → llm 链构建失败: {e}")

## 4. invoke 方法深入使用

In [ ]:
# invoke 方法深入使用 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from dotenv import load_dotenv
import os
import time

# 加载环境变量
load_dotenv()

print("⚡ invoke 方法深入使用测试:")
print("=" * 50)

openai_api_key = os.getenv('OPENAI_API_KEY')
if not openai_api_key:
    print("❌ OpenAI API Key 未配置")
else:
    try:
        # 创建测试链
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.7,
            max_tokens=40
        )
        
        prompt = PromptTemplate(
            input_variables=["item", "count"],
            template="请列出 {count} 个关于 {item} 的关键词。"
        )
        
        chain = prompt | llm
        
        print(f"📝 测试链配置:")
        print(f"   输入变量: {prompt.input_variables}")
        print(f"   模板: {prompt.template}")
        
        # 1. 基础 invoke 使用
        print(f"\n🔍 1. 基础 invoke 使用:")
        
        start_time = time.time()
        result1 = chain.invoke({"item": "编程语言", "count": 3})
        end_time = time.time()
        
        print(f"   输入: {{'item': '编程语言', 'count': 3}}")
        print(f"   执行时间: {(end_time - start_time)*1000:.1f} ms")
        print(f"   输出: {result1.content}")
        
        # 2. 不同参数组合测试
        print(f"\n🔍 2. 不同参数组合测试:")
        
        test_cases = [
            {"item": "水果", "count": 2},
            {"item": "颜色", "count": 4},
            {"item": "动物", "count": 3}
        ]
        
        for i, test_case in enumerate(test_cases, 1):
            result = chain.invoke(test_case)
            print(f"   测试 {i}: {test_case} → {len(result.content)} 字符")
        
        # 3. invoke 与其他方法对比
        print(f"\n🔍 3. invoke 与其他方法对比:")
        
        # 创建简单函数链进行对比
        def process_data(data: dict) -> str:
            return f"处理: {data.get('item', '未知')} x {data.get('count', 0)}"
        
        func_chain = RunnableLambda(process_data)
        
        # invoke 方法
        invoke_result = func_chain.invoke({"item": "测试", "count": 5})
        print(f"   invoke 结果: {invoke_result}")
        
        # 4. 错误处理和异常情况
        print(f"\n🔍 4. 错误处理和异常情况:")
        
        # 测试空输入
        try:
            empty_result = chain.invoke({})
            print(f"   空输入结果: {empty_result.content[:50]}...")
        except Exception as e:
            print(f"   空输入错误: {type(e).__name__}")
        
        # 测试无效参数类型
        try:
            type_error_result = chain.invoke("string_input")
            print(f"   字符串输入结果: {type_error_result.content[:50]}...")
        except Exception as e:
            print(f"   字符串输入错误: {type(e).__name__}")
        
        # 5. 性能测试
        print(f"\n🔍 5. 性能测试:")
        
        performance_test_cases = [
            {"item": "快速测试", "count": 1}
        ] * 3
        
        times = []
        for i, test_case in enumerate(performance_test_cases, 1):
            start_time = time.time()
            result = chain.invoke(test_case)
            end_time = time.time()
            times.append(end_time - start_time)
            print(f"   第 {i} 次: {(end_time - start_time)*1000:.1f} ms")
        
        avg_time = sum(times) / len(times)
        print(f"   平均时间: {avg_time*1000:.1f} ms")
        
        # 验证点：invoke 方法使用正确
        assert hasattr(result1, 'content'), "invoke 输出格式错误"
        assert len(result1.content) > 0, "invoke 输出为空"
        assert avg_time < 10, "invoke 执行时间过长"
        
        print(f"\n✅ 验证通过：invoke 方法深入使用正确")
        
    except Exception as e:
        print(f"\n❌ invoke 方法测试失败: {e}")

## 5. 学习总结与最佳实践

In [ ]:
# 学习总结与最佳实践 - 独立代码块
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

print("📋 Runnable 基础概念学习总结:")
print("=" * 50)

# 验证点检查
verification_points = [
    "✅ Runnable 抽象：核心接口和设计原理",
    "✅ 链式拼接：pipe 操作符和方法",
    "✅ 简单链构建：prompt → llm 组合",
    "✅ invoke 方法：统一调用接口",
]

for point in verification_points:
    print(point)

print(f"\n🎯 核心技能掌握情况: {len(verification_points)}/4 项")

print("\n💡 Runnable 最佳实践:")
print("1. 抽象理解：所有 LangChain 组件都是 Runnable")
print("2. 链式组合：使用 | 操作符进行管道连接")
print("3. 统一接口：invoke/stream/batch 方法标准化")
print("4. 函数包装：RunnableLambda 包装普通函数")
print("5. 错误处理：链中任何环节都会传播异常")

print("\n🔧 常用 Runnable 类型:")
print("1. ChatOpenAI - LLM 模型调用")
print("2. PromptTemplate - 提示词模板")
print("3. RunnableLambda - 函数包装")
print("4. RunnablePassthrough - 数据透传")
print("5. RunnableMap - 并行执行")

print("\n🚀 下一步学习建议:")
print("1. 深入学习 RunnableMap 并行执行")
print("2. 掌握 RunnablePassthrough 数据透传")
print("3. 学习 LCEL 管道语法 (prompt|llm|parser)")
print("4. 探索流式输出 Streaming")
print("5. 实践复杂链式组合应用")

# 最终验证：确保 Runnable 基础功能可用
try:
    openai_api_key = os.getenv('OPENAI_API_KEY')
    
    if openai_api_key:
        # 简单测试
        llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=30)
        prompt = PromptTemplate(
            input_variables=["text"],
            template="总结: {text}"
        )
        
        chain = prompt | llm
        result = chain.invoke({"text": "测试 Runnable"})
        
        print(f"\n🎉 最终验证成功:")
        print(f"   API 状态: 可用")
        print(f"   链类型: {type(chain).__name__}")
        print(f"   输出长度: {len(result.content)} 字符")
        print(f"   输出内容: {result.content[:50]}...")
        print("\n✅ Runnable 基础概念学习完成！")
        
        print(f"\n🎊 LCEL 编排层学习开始！")
        print(f"   已掌握: Runnable 抽象和基础链式组合")
        print(f"   下一步: 学习并行执行和高级组合技术")
        
    else:
        print(f"\n⚠️  最终验证跳过: OpenAI API Key 未配置")
        print("   请配置 OPENAI_API_KEY 后重试")
        
except Exception as e:
    print(f"\n❌ 最终验证失败: {e}")